In [2]:
pip install optuna

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.3/365.3 KB 7.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 210.6/210.6 KB 20.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 78.7/78.7 KB 10.8 MB/s eta 0:00:00


In [3]:
from sklearn.metrics import mean_absolute_percentage_error
import optuna
from optuna.integration import XGBoostPruningCallback
#import xgboost as xgb
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from matplotlib import mlab
import matplotlib.pyplot as plt
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory
from google.colab import drive
drive.mount('/content/drive')
import os
for dirname, _, filenames in os.walk('/content/drive/MyDrive/godaddy-microbusiness-density-forecasting'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
sample_submission = pd.read_csv(r'/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/sample_submission.csv')
test = pd.read_csv(r'/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/test.csv')
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Mounted at /content/drive
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/sample_submission.csv
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/census_starter.csv
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/test.csv
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/epcoh1000.data-00000-of-00001
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/checkpoint
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/train.csv
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/prepared_train_data.csv
/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/submission.csv


In [4]:
!pip uninstall xgboost

Found existing installation: xgboost 0.90
Uninstalling xgboost-0.90:
  Would remove:
    /usr/local/lib/python3.8/dist-packages/xgboost-0.90.dist-info/*
    /usr/local/lib/python3.8/dist-packages/xgboost/*
    /usr/local/xgboost/libxgboost.so
Proceed (Y/n)? y
  Successfully uninstalled xgboost-0.90


In [5]:
!pip install xgboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 MB 5.7 MB/s eta 0:00:00


In [6]:
import xgboost as xgb
xgb.__version__

'0.90'

In [8]:
#Train Data
train = pd.read_csv(r'/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/train.csv')
train['first_day_of_month'] = pd.to_datetime(train['first_day_of_month'])
#extract date from row_id
date = train['row_id'].str.split('_').apply(lambda x:x[1])
year = date.str.split('-').apply(lambda x:x[0])
train['year'] = year
train['year'] = train['year'].astype(int)
#give order to each cfipgs by it's sequence
train['order'] = train.groupby(['cfips'])['row_id'].cumcount()
#Calculate microbusiness_density(Rate of Change) -> price(time n) / price(time n-1)
train['last_month_density'] = train.groupby('cfips')['microbusiness_density'].shift(1)
train.loc[train[train['order']!=38].index, 'last_month_density'].fillna(1, inplace=True)
train.loc[train[train['order']!=38].index, 'last_month_density'].replace(0,1, inplace=True)
train.loc[train[(train['microbusiness_density']!=0)&(train['last_month_density']==0)&(train['order']!=38)].index, 'last_month_density'] = 1
train['target'] =  train['microbusiness_density'] / train['last_month_density']
train = train.reset_index(drop=True)
train.loc[train[(train['order']!=38)&(train['target'].isnull())].index,'target'] = 1
train.loc[train.target.isnull(), 'target'] = 1
train['state'] = train['state'].astype('category')
train['county'] = train['county'].astype('category')
train

,row_id,cfips,county,state,first_day_of_month,microbusiness_density,active,year,order,last_month_density,target
0,1001_2019-08-01,1001,Autauga County,Alabama,2019-08-01,3.007682,1249,2019,0,NaN,1.000000
1,1001_2019-09-01,1001,Autauga County,Alabama,2019-09-01,2.884870,1198,2019,1,3.007682,0.959167
2,1001_2019-10-01,1001,Autauga County,Alabama,2019-10-01,3.055843,1269,2019,2,2.884870,1.059265
3,1001_2019-11-01,1001,Autauga County,Alabama,2019-11-01,2.993233,1243,2019,3,3.055843,0.979511
4,1001_2019-12-01,1001,Autauga County,Alabama,2019-12-01,2.993233,1243,2019,4,2.993233,1.000000
...,...,...,...,...,...,...,...,...,...,...,...
122260,56045_2022-06-01,56045,Weston County,Wyoming,2022-06-01,1.803249,101,2022,34,1.803249,1.000000
122261,56045_2022-07-01,56045,Weston County,Wyoming,2022-07-01,1.803249,101,2022,35,1.803249,1.000000
122262,56045_2022-08-01,56045,Weston County,Wyoming,2022-08-01,1.785395,100,2022,36,1.803249,0.990099
122263,56045_2022-09-01,56045,Weston County,Wyoming,2022-09-01,1.785395,100,2022,37,1.785395,1.000000


In [ ]:
#Census data from Census Bureau's American Community Survey (ACS)
#Suspended!
census_starter = pd.read_csv(r'/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/census_starter.csv')
census_starter_data = pd.DataFrame(columns=['cfips','year'])
for subject in ['pct_bb','pct_college','pct_foreign_born','pct_it_workers','median_hh_inc']:
    subject_list = [i for i in census_starter.columns if(subject in i)]
    subject_list.insert(0,'cfips')
    subject_tb = census_starter[subject_list].melt(id_vars=["cfips"], var_name="type", value_name=subject)
    subject_tb['year'] = subject_tb['type'].str.split('_').apply(lambda x :x[-1])
    subject_tb = subject_tb[['cfips', 'year', subject]]
    census_starter_data = census_starter_data.merge(subject_tb, on=['cfips', 'year'], how='outer')
#last year census starter mapped to current year of train data
census_starter_data['year'] = (census_starter_data['year'].astype(int)+1)
#Calculate customized rate of change(ROC) for each column in census_starter
def census_diff_func(data, column_lst):
  for column in column_lst:
    column_lag = data.groupby('cfips')[column].shift(1)
    data[f'{column}_diff'] = abs(data[column] - column_lag)/(data[column] + column_lag)
  return data
census_lst = ['pct_bb', 'pct_college', 'pct_foreign_born','pct_it_workers', 'median_hh_inc']
census_starter_data = census_diff_func(census_starter_data, census_lst)
#train = train.merge(census_starter_data, on=['cfips', 'year'], how='left')
print('census_starter:',census_starter_data.year.min())
print('census_starter:',census_starter_data.year.max())

census_starter: 2018
census_starter: 2022


In [9]:
#Moving Average of the features : SMA/CMA/EMA
#from time t to (t - num)
def mv_avg(data, column, num):
  for lag_num in range(1,6):
    data[f'microbusiness_density_log_lag_{lag_num}'] = np.log(data.groupby('cfips')['microbusiness_density'].shift(lag_num)+1)
  for lag_num in range(1,3):
    data[f'{column}_lag_{lag_num}'] = data.groupby('cfips')[column].shift(lag_num)
  for lag_num in range(2,3):
    data[f'{column}_diff_{lag_num-1}'] = abs(data[f'{column}_lag_{lag_num-1}'] - data[f'{column}_lag_{lag_num}'])/ \
                                            (data[f'{column}_lag_{lag_num-1}'] + data[f'{column}_lag_{lag_num}'])

  column_ema = data.groupby('cfips')[column].ewm(span=num).mean()
  data[f'{column}_EMA'] = column_ema.rename(f'{column}_EMA').reset_index(level=0)[f'{column}_EMA']
  data[f'{column}_EMA_lag'] = data.groupby('cfips')[f'{column}_EMA'].shift(1)
  data[f'{column}_EMA_diff'] = abs(data[f'{column}_lag_1'] - data[f'{column}_EMA_lag'])/(data[f'{column}_lag_1'] + data[f'{column}_EMA_lag'])
  return data
num = 5
train = mv_avg(train, 'target', num)

In [14]:
#First Step cleansing
#Overwrite target values over lower and upper bounds with corresponding EMA target values(time t-1)
lower_bound_percentile = .0125
lower_bound = train['target'].quantile([lower_bound_percentile])[lower_bound_percentile]
upper_bound_percentile = .9875
upper_bound = train['target'].quantile([upper_bound_percentile])[upper_bound_percentile]

outlier = train[(train['target']>upper_bound)|(train['target']<lower_bound)]['cfips'].unique()
drop_lst = []
for cfips in outlier:
  cfips_data = train[train['cfips']==cfips][['target', 'target_EMA_lag']]
  idx_lst = cfips_data[(cfips_data['target']>upper_bound)|(cfips_data['target']<lower_bound)].index
  for idx in list(idx_lst)[::-1]:
    if idx == cfips_data.index[0]:
      train.drop(idx, inplace=True)
      drop_lst.append(idx)
      target = train.loc[idx, 'target']
      ma = cfips_data.loc[idx-1, 'target_EMA_lag']
      if ma > upper_bound:
        ma = upper_bound
      elif ma < lower_bound:
        ma = lower_bound
      train.loc[idx, 'target'] = ma

In [15]:
def plot_cfips(cfips = None):
    plt.figure(figsize = (15,4))
    plot_df = train[train['cfips'] == cfips]
    plt.plot(plot_df['first_day_of_month'], plot_df['target'], '-o', label = 'train data')
    plt.title('Microbussiness Trend Analysis CFIPS: ' + str(cfips), size = 12)
    plt.xlabel('Date',size = 5)
    plt.ylabel('Microbusiness Density',size = 12)
    plt.legend()
    plt.grid(alpha = 0.5)
    plt.show()
    return None

In [17]:
#cfips在target_diff總和仍過2，進第二輪清理
def second_step_cleansing(data, lower_bound, upper_bound):
  target_diff_sum = data.groupby('cfips')['target_diff_1'].sum().rename('target_diff_sum').reset_index()
  target_diff_sum = target_diff_sum.loc[target_diff_sum.target_diff_sum>2, 'cfips'].tolist()
  check = data[(data['cfips'].isin(target_diff_sum))&(data['target']>=lower_bound)&(data['target']<=upper_bound)]
  mean = check.groupby('cfips')['target'].mean().rename('target_mean').reset_index()
  check = check.merge(mean)
  idx = (check['cfips'].value_counts()>=16)
  cleansed_list = list(idx[idx].index)
  conditioned_train = data[(data['cfips'].isin(cleansed_list))&(data['order']!=38)]
  conditioned_train = conditioned_train[(conditioned_train['target']>upper_bound)|(conditioned_train['target']<lower_bound)]
  original_idx = conditioned_train.index
  merged_train_mean = conditioned_train.merge(mean, how='left')
  merged_train_mean.index = original_idx
  data.loc[original_idx, 'target'] = merged_train_mean['target_mean']
  return data, target_diff_sum
train, target_diff_sum = second_step_cleansing(train, 0.99, 1.01)

In [20]:
def smape(y_true, y_pred):
    zero_idx = (y_true==0) & (y_pred==0)
    y_true[zero_idx] = 1
    y_pred[zero_idx] = 1
    score = 1 / len(y_true) * np.sum(2 * np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred)) * 100)
    return score

In [21]:
feature = ['target_diff_1','microbusiness_density_log_lag_1','microbusiness_density_log_lag_2', 'microbusiness_density_log_lag_3']
val = train.query("first_day_of_month == '2022-10-01'")
val_x, val_y = val[feature], val['target']

In [22]:
from optuna.integration import LightGBMPruningCallback
import lightgbm as lgb

In [46]:
def objective(trial, data, val_x, val_y, val, feature):
  params = {
        'n_iter': trial.suggest_int('n_iter', 1000, 2000, 50),
        'verbosity': -1,
        'objective': 'mape',
        'random_state': 42,
        'extra_trees': True,
        'colsample_bytree': trial.suggest_float('colsample_bytree', 0.1, 1.0),
        'colsample_bynode': trial.suggest_float('colsample_bynode', 0.1, 1.0),
        'max_depth': trial.suggest_int('max_depth', 3, 10),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.1, log=True),
        'lambda_l1': trial.suggest_float('lambda_l1', 1e-2, 10.0),
        'lambda_l2': trial.suggest_float('lambda_l2', 1e-2, 10.0),
        'num_leaves': trial.suggest_int('num_leaves', 8, 1024),
        'min_data_in_leaf': trial.suggest_int('min_data_in_leaf', 5, 250),}
    # LGBM建模
  train_data = data[~data['order'].isin([0,1,2,3,4,38])]   
  train_data = train_data[train_data['order']!=38]
  model = lgb.LGBMRegressor(**params)
  model.fit(
        train_data[feature],
        train_data['target'],
        eval_set=[(val_x, val_y)],
        eval_metric="mape",
        early_stopping_rounds=10,
        callbacks=[
            LightGBMPruningCallback(trial, "mape")
        ],
    )
    # 模型预测
  pred = model.predict(val_x)
  val = val.assign(pred=pred)
    #
  val = val.assign(pred_density=lambda df: df.pred*df.last_month_density.to_numpy())
    #val = val.assign(pred_density=lambda df: df.apply(lambda x: x.pred*x.last_month_density.to_numpy()))
    # print(mean_absolute_percentage_error(val['microbusiness_density'], val['pred_density']))
  score = smape(val['microbusiness_density'], val['pred_density'])

  return score

In [47]:
study = optuna.create_study(direction="minimize", study_name="LGBM Regressor")
func = lambda trial: objective(trial, train, val_x, val_y, val,feature)
study.optimize(func, n_trials=1000)

[I 2023-02-10 15:50:36,511] A new study created in memory with name: LGBM Regressor


[1]	valid_0's mape: 0.0105089
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's mape: 0.010505
[3]	valid_0's mape: 0.0105017
[4]	valid_0's mape: 0.0105013
[5]	valid_0's mape: 0.0104978
[6]	valid_0's mape: 0.010495
[7]	valid_0's mape: 0.0104909
[8]	valid_0's mape: 0.0104909
[9]	valid_0's mape: 0.0104879
[10]	valid_0's mape: 0.0104851
[11]	valid_0's mape: 0.0104824
[12]	valid_0's mape: 0.0104823
[13]	valid_0's mape: 0.0104794
[14]	valid_0's mape: 0.0104769
[15]	valid_0's mape: 0.0104746
[16]	valid_0's mape: 0.0104746
[17]	valid_0's mape: 0.0104721
[18]	valid_0's mape: 0.0104701
[19]	valid_0's mape: 0.0104681
[20]	valid_0's mape: 0.0104681
[21]	valid_0's mape: 0.0104665
[22]	valid_0's mape: 0.0104647
[23]	valid_0's mape: 0.010463
[24]	valid_0's mape: 0.010463
[25]	valid_0's mape: 0.0104617
[26]	valid_0's mape: 0.0104603
[27]	valid_0's mape: 0.0104589
[28]	valid_0's mape: 0.0104589
[29]	valid_0's mape: 0.0104574
[30]	valid_0's mape: 0.0104563
[31]	valid_0's mape: 

<ipython-input-20-379ca982565c>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_true[zero_idx] = 1
<ipython-input-20-379ca982565c>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  y_pred[zero_idx] = 1
[I 2023-02-10 15:50:44,323] Trial 0 finished with value: 1.091532350204454 and parameters: {'n_iter': 1650, 'colsample_bytree': 0.3361623311878659, 'colsample_bynode': 0.9781750031703957, 'max_depth': 4, 'learning_rate': 0.03693923837515496, 'lambda_l1': 1.0349145429445892, 'lambda_l2': 2.7068595439487586, 'num_leaves': 199, 'min_data_in_leaf': 147}. Best is trial 0 with value: 1.091532350204454.


[1]	valid_0's mape: 0.010505
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's mape: 0.0104986
[3]	valid_0's mape: 0.0104915
[4]	valid_0's mape: 0.0104899
[5]	valid_0's mape: 0.0104839
[6]	valid_0's mape: 0.0104784
[7]	valid_0's mape: 0.0104731
[8]	valid_0's mape: 0.010472
[9]	valid_0's mape: 0.0104671
[10]	valid_0's mape: 0.010463
[11]	valid_0's mape: 0.0104594
[12]	valid_0's mape: 0.0104591
[13]	valid_0's mape: 0.0104554
[14]	valid_0's mape: 0.0104525
[15]	valid_0's mape: 0.0104494
[16]	valid_0's mape: 0.0104492
[17]	valid_0's mape: 0.0104462
[18]	valid_0's mape: 0.0104432
[19]	valid_0's mape: 0.0104406
[20]	valid_0's mape: 0.01044
[21]	valid_0's mape: 0.0104377
[22]	valid_0's mape: 0.0104355
[23]	valid_0's mape: 0.0104336
[24]	valid_0's mape: 0.0104331
[25]	valid_0's mape: 0.0104312
[26]	valid_0's mape: 0.0104294
[27]	valid_0's mape: 0.0104282
[28]	valid_0's mape: 0.0104284
[29]	valid_0's mape: 0.0104267
[30]	valid_0's mape: 0.0104256
[31]	valid_0's mape: 0

[I 2023-02-10 15:50:46,100] Trial 1 finished with value: 1.0903843382959935 and parameters: {'n_iter': 1850, 'colsample_bytree': 0.4310181533644466, 'colsample_bynode': 0.9642598826090063, 'max_depth': 7, 'learning_rate': 0.06796385946769913, 'lambda_l1': 4.819546696038488, 'lambda_l2': 4.631512317059268, 'num_leaves': 426, 'min_data_in_leaf': 78}. Best is trial 1 with value: 1.0903843382959935.


...

[I 2023-02-10 15:58:11,399] Trial 975 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:11,726] Trial 976 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:12,059] Trial 977 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:12,413] Trial 978 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:12,768] Trial 979 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:13,113] Trial 980 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:13,453] Trial 981 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:13,774] Trial 982 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:14,104] Trial 983 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:14,464] Trial 984 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:14,794] Trial 985 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:15,138] Trial 986 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:15,532] Trial 987 pruned. Trial 

[1]	valid_0's mape: 0.0105015
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's mape: 0.0104908
[3]	valid_0's mape: 0.0104819


[I 2023-02-10 15:58:15,868] Trial 988 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:16,201] Trial 989 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:16,564] Trial 990 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:16,914] Trial 991 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:17,257] Trial 992 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:17,588] Trial 993 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:17,930] Trial 994 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:18,263] Trial 995 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:18,603] Trial 996 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:19,051] Trial 997 pruned. Trial was pruned at iteration 5.


[1]	valid_0's mape: 0.0105013
Training until validation scores don't improve for 10 rounds.
[2]	valid_0's mape: 0.0104906
[3]	valid_0's mape: 0.0104804
[4]	valid_0's mape: 0.010474
[5]	valid_0's mape: 0.010468


[I 2023-02-10 15:58:19,410] Trial 998 pruned. Trial was pruned at iteration 0.
[I 2023-02-10 15:58:19,746] Trial 999 pruned. Trial was pruned at iteration 0.


In [48]:
print(f"\tBest value (smape): {study.best_value:.5f}")
print(f"\tBest params:")

for key, value in study.best_params.items():
    print(f"\t\t{key}: {value}")

	Best value (smape): 1.08907
	Best params:
		n_iter: 1900
		colsample_bytree: 0.6215546834772154
		colsample_bynode: 0.23239662667518524
		max_depth: 5
		learning_rate: 0.09976342368898171
		lambda_l1: 1.815505646656365
		lambda_l2: 1.0602975835829564
		num_leaves: 631
		min_data_in_leaf: 98


In [49]:
model  = lgb.LGBMRegressor(boosting='gbdt', objective='mape',
                           random_state=42, metric='mape',
                           **study.best_params)
model

LGBMRegressor(boosting='gbdt', colsample_bynode=0.23239662667518524,
              colsample_bytree=0.6215546834772154, lambda_l1=1.815505646656365,
              lambda_l2=1.0602975835829564, learning_rate=0.09976342368898171,
              max_depth=5, metric='mape', min_data_in_leaf=98, n_iter=1900,
              num_leaves=631, objective='mape', random_state=42)

In [28]:
#param = {'objective'           :'reg:pseudohubererror',         
#        'tree_method'          :'hist',
#        'n_estimators'         : 5000,
#        'learning_rate'        : 0.001,
#        'max_leaves'           : 32,
#        'subsample'            : 0.50,
#       'colsample_bytree'     : 0.50,
#        'max_bin'              : 4096,
#        'n_jobs'               : -1,
#        'eval_metric'          :'mae',
        #'early_stopping_rounds': 128
#        }
#model = xgb.XGBRegressor(**param, enable_categorical=True)
#model

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.5, early_stopping_rounds=None,
             enable_categorical=True, eval_metric='mae', feature_types=None,
             gamma=None, gpu_id=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.001, max_bin=4096,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=32,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             n_estimators=5000, n_jobs=-1, num_parallel_tree=None,
             objective='reg:pseudohubererror', predictor=None, ...)

In [33]:
feature = ['target_diff_1','microbusiness_density_log_lag_1','microbusiness_density_log_lag_2', 'microbusiness_density_log_lag_3']

In [50]:
def model_method(data, model, feature):
  train_data = data[~data['order'].isin([0,1,2,3,4])]
  model.fit(train_data[feature], train_data['target'], verbose=False)
  feature_importance = pd.DataFrame({'column':feature,'score':model.feature_importances_})
  return model, feature_importance, feature

In [51]:
model, feature_importance, feature = model_method(train, model, feature)

In [52]:
#Feature Importance
feature_importance

,column,score
0,target_diff_1,11464
1,microbusiness_density_log_lag_1,12219
2,microbusiness_density_log_lag_2,8839
3,microbusiness_density_log_lag_3,10424


#Submission

In [53]:
base = train[['cfips', 'order', 'target', 'target_lag_1','target_lag_2','microbusiness_density']+feature].copy()
base = base.reset_index(drop=True)
prediction = base[base['order']==38].reset_index(drop=True)
order = 39
prediction['microbusiness_density_log_lag_3'] = prediction['microbusiness_density_log_lag_2']
prediction['microbusiness_density_log_lag_2'] = prediction['microbusiness_density_log_lag_1']
prediction['microbusiness_density_log_lag_1'] = np.log(prediction['microbusiness_density']+1)
target_lag_2 = prediction['target_lag_1'].to_numpy()
target_lag_1 = prediction['target'].to_numpy()
target_diff_1 = abs(target_lag_1 - target_lag_2)/(target_lag_1 + target_lag_2)
prediction['target_diff_1'] = target_diff_1
prediction['order'] = order
target = model.predict(prediction[feature])
prediction['target'] = target
base = base.append(prediction)

,cfips,order,target,target_lag_1,target_lag_2,microbusiness_density,target_diff_1,microbusiness_density_log_lag_1,microbusiness_density_log_lag_2,microbusiness_density_log_lag_3
0,1001,38,1.006152,1.005498,0.995893,3.463856,0.004799,1.491257,1.487011,1.490197
1,1003,38,1.002170,0.982399,0.990399,8.359798,0.004055,2.234488,2.250360,2.258995
2,1005,38,1.020921,1.008439,0.983403,1.232074,0.012569,0.791556,0.786969,0.796122
3,1007,38,0.978633,1.017391,0.974576,1.287240,0.021494,0.839559,0.829800,0.844403
4,1009,38,0.989051,1.008589,1.002460,1.831783,0.003048,1.048042,1.042496,1.040906
...,...,...,...,...,...,...,...,...,...,...
3130,56037,38,1.004440,1.000000,0.991199,2.823801,0.004420,1.337976,1.337976,1.344504
3131,56039,38,1.007000,1.000200,1.005633,26.273220,0.002708,3.299186,3.298994,3.293585
3132,56041,38,1.003448,1.005199,0.998270,4.009369,0.003459,1.608556,1.604410,1.605794
3133,56043,38,0.974227,1.000000,0.994872,3.126551,0.002571,1.437288,1.437288,1.441210


In [54]:
for order in range(40, 47):
  target = model.predict(prediction[feature])
  prediction['target'] = target
  prediction['microbusiness_density_log_lag_3'] = prediction['microbusiness_density_log_lag_2']
  prediction['microbusiness_density_log_lag_2'] = prediction['microbusiness_density_log_lag_1']
  prediction['microbusiness_density_log_lag_1'] = np.log(prediction['microbusiness_density']+1)  
  prediction['microbusiness_density'] = target*np.array(prediction['microbusiness_density'])
  target_lag_2 = base[base['order']==(order-1)]['target_lag_1'].to_numpy()
  target_lag_1 = base[base['order']==(order-1)]['target'].to_numpy()
  target_diff_1 = abs(target_lag_1 - target_lag_2)/(target_lag_1 + target_lag_2)
  prediction['target_diff_1'] = target_diff_1
  prediction['order'] = order
  base = base.append(prediction)
  print('prediction \n', prediction)
base = base[base['order']>=39].reset_index(drop=True)
base['row_id'] = test['row_id']
base[['row_id', 'microbusiness_density']].to_csv(r'/content/drive/MyDrive/godaddy-microbusiness-density-forecasting/submission.csv', index=False)

prediction 
       cfips  order    target  target_lag_1  target_lag_2  \
0      1001     40  1.000075      1.005498      0.995893   
1      1003     40  1.003475      0.982399      0.990399   
2      1005     40  1.000590      1.008439      0.983403   
3      1007     40  1.000000      1.017391      0.974576   
4      1009     40  1.000000      1.008589      1.002460   
...     ...    ...       ...           ...           ...   
3130  56037     40  1.000000      1.000000      0.991199   
3131  56039     40  1.001893      1.000200      1.005633   
3132  56041     40  1.001766      1.005199      0.998270   
3133  56043     40  1.000000      1.000000      0.994872   
3134  56045     40  1.000000      1.000000      0.990099   

      microbusiness_density  target_diff_1  microbusiness_density_log_lag_1  \
0                  3.464114   2.704322e-03                         1.496013   
1                  8.388847   1.061267e-02                         2.236424   
2                  1.232802  